In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# %cd /content/drive/MyDrive/Mark1
%cd /content/drive/MyDrive/B.Tech./4th year/sem1/AML/Project/Mark1

/content/drive/MyDrive/B.Tech./4th year/sem1/AML/Project/Mark1


In [ ]:
!ls

Bad_teacher1.pt		 Mark1.ipynb
Bad_teacher2.pt		 Mark2.ipynb
Bad_teacher.pt		 metrics.py
cifar-100-python	 model.py
cifar-100-python.tar.gz  __pycache__
cifar-10-batches-py	 ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt
cifar-10-python.tar.gz	 unlearn.py
dataset.py		 utils.py


In [ ]:
!pip install transformers

In [ ]:
import torch

from dataset import *
from model import ResNet18
from unlearn import *
from metrics import UnLearningScore
from utils import *
from torch.utils.data import DataLoader

# import required libraries
import numpy as np
import tarfile
import os

from torch import nn
import torch.nn.functional as F
# from torchvision.datasets.utils import download_url
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torchvision.models import resnet18

torch.manual_seed(100)

In [ ]:
transform_train = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = tt.Compose([
    tt.ToTensor(),
    tt.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
train_ds = CustomCIFAR10(root='.', train=True,download=True, transform=transform_train)
valid_ds = CustomCIFAR10(root='.', train=False,download=True, transform=transform_train)

batch_size = 256
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=1, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=1, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
len(train_ds)

50000

In [ ]:
from torch.utils.data import Subset
import random
forget_classes = [0]

# Create a Subset for forget and retain classes in the train dataset
forget_train_indices = [idx for idx, (_, label, _) in enumerate(train_ds) if label in forget_classes]
retain_train_indices = [idx for idx, (_, label, _) in enumerate(train_ds) if label not in forget_classes]

# Create a Subset for forget and retain classes in the valid dataset
forget_valid_indices = [idx for idx, (_, label, _) in enumerate(valid_ds) if label in forget_classes]
retain_valid_indices = [idx for idx, (_, label, _) in enumerate(valid_ds) if label not in forget_classes]


# Create DataLoaders for forget and retain categories in the train dataset
forget_train = Subset(train_ds, forget_train_indices)
retain_train = Subset(train_ds, retain_train_indices)

# Create DataLoaders for forget and retain categories in the valid dataset
forget_valid = Subset(valid_ds, forget_valid_indices)
retain_valid = Subset(valid_ds, retain_valid_indices)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=2, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=2, pin_memory=True, shuffle=True)

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=2, pin_memory=True)
retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=2, pin_memory=True)



In [ ]:
retain_train_subset_indices = random.sample(retain_train_indices, int(1*len(retain_train_indices)))
retain_train_subset = Subset(train_ds, retain_train_subset_indices)
retain_train_subset_dl = DataLoader(retain_train_subset, batch_size, num_workers=2, pin_memory=True, shuffle=True)

In [ ]:
len(retain_train_subset_indices)

13500

# UNSIR

## Train/Load the model

In [ ]:
device = "cuda:0"
model = resnet18(num_classes = 10).to(device = device)

epochs = 20
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time
history = fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl,device,
                             grad_clip=grad_clip,
                             weight_decay=weight_decay,
                             opt_func=opt_func)

torch.save(model.state_dict(), "ResNET18_CIFAR10_ALL_CLASSES_Exp1_fc_0.pt")

In [ ]:
device = 'cuda'
model = resnet18(num_classes = 10).to(device = device)
model.load_state_dict(torch.load("ResNET18_CIFAR10_ALL_CLASSES_Exp1_fc_0.pt", map_location = device))

<All keys matched successfully>

## Curriculum create

In [ ]:
all_confidence_scores = []
with torch.no_grad():
    for batch in retain_train_subset_dl:
        images,_,_ = batch
        images = images.to(device)
        output = model(images)
        confidence_scores = torch.nn.functional.softmax(output, dim=1)
        confidence_scores = confidence_scores.max(dim=1)
        confidence_scores = confidence_scores[0].tolist()
        print(confidence_scores)
        all_confidence_scores.extend(confidence_scores)

# Sort indices based on predictions
sorted_indices = [index for _, index in sorted(zip(all_confidence_scores, retain_train_subset_indices), reverse=True)]

# Now, you can create a new subset using the sorted indices
sorted_retain_train_subset = Subset(train_ds, sorted_indices)

In [ ]:
len(retain_train_subset)

45000

## Noise learn

In [ ]:
noise = UNSIR_noise(batch_size, 3, 32, 32).to(device)

noise_batch_size = 256
forget_class_label = 0
num_epochs = 5

noise =  UNSIR_noise_train(noise, model, forget_class_label, num_epochs,\
                           noise_batch_size, device=device)

Loss: 185.71633911132812
Loss: 36.847206115722656
Loss: 0.7868863344192505
Loss: -6.738899230957031
Loss: -10.127561569213867


In [ ]:
def UNSIR_create_noisy_loader_(noise, forget_class_label, retain_samples, batch_size, num_noise_batches=80, device='cuda'):
    print("Noisy_Loader Called")
    noisy_data = []
    for i in range(num_noise_batches):
        batch = noise()
        for i in range(batch[0].size(0)):
            noisy_data.append((batch[i].detach().cpu(), torch.tensor(forget_class_label), \
                               torch.tensor(forget_class_label)))

    len_retain = len(retain_samples)
    len_noise = len(noisy_data)

    ration_r_n = int(len_retain/len_noise)

    num = 0
    j = 0
    total_samples = []
    for i in range(len(retain_samples)):
        if(num%ration_r_n == 0):
            total_samples.append(noisy_data[j])
            j += 1
            j = j%len_noise
        total_samples.append((retain_samples[i][0].cpu(), torch.tensor(retain_samples[i][2]),\
                            torch.tensor(retain_samples[i][2])))
        num += 1
    # noisy_data += other_samples
    noisy_loader = torch.utils.data.DataLoader(total_samples, batch_size=batch_size, shuffle = False)

    return noisy_loader


In [ ]:

noisy_loader_curriculum = UNSIR_create_noisy_loader_(noise, forget_class_label\
                                         , sorted_retain_train_subset, batch_size, device=device)

noisy_loader_no_curriculum = UNSIR_create_noisy_loader_(noise, forget_class_label\
                                         , retain_train_subset, batch_size, device=device)

Noisy_Loader Called
Noisy_Loader Called


## Impair

In [ ]:
#impair step
def impair(noisy_loader, model):
    epochs = 1
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.02)
    history = []
    for epoch in range(epochs):
        model.train(True)
        train_losses = []
        lrs = []
        for batch in noisy_loader:
            images, labels, clabels = batch
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            out = model(images)                  # Generate predictions
            loss = F.cross_entropy(out, labels) # Calculate loss
            loss.backward()
            train_losses.append(loss.detach().cpu())

            optimizer.step()

            lrs.append(get_lr(optimizer))

        result = evaluate(model, retain_valid_dl, device)
        result['train_loss'] = torch.stack(train_losses).mean()
        result['lrs'] = lrs
        epoch_end(model, epoch, result)
        history.append(result)
# history, bad_teacher = fit_one_unlearning_cycle(epochs, model, noisy_loader, retain_valid_dl, device = device, lr = 0.02)

In [ ]:
device = 'cuda'
model1 = resnet18(num_classes = 10).to(device = device)
model1.load_state_dict(torch.load("ResNET18_CIFAR10_ALL_CLASSES_Exp1_fc_0.pt", map_location = device))
impair(noisy_loader_curriculum, model1)
torch.save(model1.state_dict(), "Bad_teacher_Exp2_fc0_with_curriculum.pt")


model2 = resnet18(num_classes = 10).to(device = device)
model2.load_state_dict(torch.load("ResNET18_CIFAR10_ALL_CLASSES_Exp1_fc_0.pt", map_location = device))
impair(noisy_loader_no_curriculum, model2)
torch.save(model2.state_dict(), "Bad_teacher_Exp2_fc0_without_curriculum.pt")

Epoch [0], last_lr: 0.02000, train_loss: 0.7327, val_loss: 1.1610, val_acc: 56.5517
Epoch [0], last_lr: 0.02000, train_loss: 0.7152, val_loss: 0.9392, val_acc: 68.8129


In [ ]:
# load the trained model
device = 'cuda'
good_teacher = resnet18(num_classes = 10).to(device = device)
good_teacher.load_state_dict(torch.load("ResNET18_CIFAR10_ALL_CLASSES_Exp1_fc1.pt", map_location='cuda'))

# load the trained model
device = 'cuda'
bad_teacher1 = resnet18(num_classes = 10).to(device = device)
bad_teacher1.load_state_dict(torch.load("Bad_teacher_Exp2_fc0_with_curriculum.pt", map_location='cuda'))

device = 'cuda'
bad_teacher2 = resnet18(num_classes = 10).to(device = device)
bad_teacher2.load_state_dict(torch.load("Bad_teacher_Exp2_fc0_without_curriculum.pt", map_location='cuda'))

<All keys matched successfully>

In [ ]:
print("Original Model")
print(evaluate(good_teacher, retain_valid_dl, device))

print("Unlearned Model")
print(evaluate(bad_teacher, retain_valid_dl, device))

Original Model
{'Loss': 1.3840818405151367, 'Acc': 76.15885162353516}
Unlearned Model
{'Loss': 0.8831408023834229, 'Acc': 69.00390625}


In [ ]:
%%time
print("Original Model")
print(evaluate(good_teacher, retain_train_dl, device))

print("Unlearned Model")
print(evaluate(bad_teacher, retain_train_dl, device))

Original Model
{'Loss': 0.0007699645939283073, 'Acc': 99.99333953857422}
Unlearned Model
{'Loss': 0.711523711681366, 'Acc': 75.3733139038086}
CPU times: user 4.88 s, sys: 611 ms, total: 5.49 s
Wall time: 25.9 s


In [ ]:
print("Original Model")
print(evaluate(good_teacher, forget_valid_dl, device))

print("Unlearned Model")
print(evaluate(bad_teacher, forget_valid_dl, device))


Original Model
{'Loss': 1.0920532941818237, 'Acc': 81.02101135253906}
Unlearned Model
{'Loss': 12.127893447875977, 'Acc': 0.0}


## Repair Step

In [ ]:
optimizer = torch.optim.Adam(bad_teacher.parameters(), lr = 0.01)

data_loader = DataLoader(sorted_retain_train_subset, batch_size=256, num_workers=2, pin_memory=True, shuffle=False)
for epoch in range(1):
    bad_teacher.train(True)
    running_loss = 0.0
    running_acc = 0
    for batch in data_loader:
        images, labels, clabels = batch
        inputs, labels = images.to(device), labels.to(device)
        # inputs, labels = data
        # inputs, labels = inputs.cuda(),torch.tensor(labels).cuda()

        optimizer.zero_grad()
        outputs = bad_teacher(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==labels.shape
        running_acc += (labels==out).sum().item()
    print(f"Train loss {epoch+1}: {running_loss/len(train_ds)},Train Acc:{running_acc*100/len(train_ds)}%")

Train loss 1: 0.3991092208766937,Train Acc:76.03%


In [ ]:
print("Unlearned Model")
print(evaluate(bad_teacher, retain_valid_dl, device))

Unlearned Model
{'Loss': 0.8127613663673401, 'Acc': 72.23958587646484}


In [ ]:
device = 'cuda'
# unlearning_teacher = ResNet18(num_classes = 20, pretrained = False).to(device).eval()
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))
good_teacher = good_teacher.eval()

KL_temperature = 1

optimizer = torch.optim.Adam(student_model.parameters(), lr = 0.0001)

blindspot_unlearner(model = student_model, unlearning_teacher = bad_teacher, full_trained_teacher = good_teacher,
          retain_data = retain_train_subset, forget_data = forget_train, epochs = 1, optimizer = optimizer, lr = 0.0001,
          batch_size = 256, num_workers = 2, device = device, KL_temperature = KL_temperature)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 1 Unlearning Loss 0.008439322002232075


In [ ]:
# performance of unlearned model on forget set
evaluate(student_model, forget_valid_dl, device)

{'Loss': 5.754669666290283, 'Acc': 2.0}

In [ ]:
# performance of unlearned model on retain set
evaluate(student_model, retain_valid_dl, device)

{'Loss': 0.6966467499732971, 'Acc': 80.03782653808594}

In [ ]:
print("Initial Score: {}".format(UnLearningScore(good_teacher, bad_teacher, forget_valid_dl, 256, 'cuda')))
print("Our Score: {}".format(UnLearningScore(student_model, bad_teacher, forget_valid_dl, 256, 'cuda')))
# print("Gold Score: {}".format(UnLearningScore(gold_model, bad_teacher, forget_valid_dl, 256, 'cuda')))
# print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, 'cuda')))

Initial Score: -inf


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Our Score: -inf
